# TOP 
- 只是用來把所有xml檔收集起來 整理成dataframe

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from glob import glob
%matplotlib inline
import cv2
from tqdm import tqdm_notebook as tqdm

In [2]:
from pathlib import Path

In [3]:
base = Path('/home/jovyan/AOI/data/')
data1_path = base / '20180703'
data1_label_path = data1_path / 'label'
data2_path = base / '20180705'
data2_label_path = data1_path / 'label'
data3_path = base / '20180726_S1'

In [4]:
from helper import read_files2df

In [5]:
files = glob(str(data3_path / 'final' / '*' / '*.xml'))
df = read_files2df(files)
files = glob(str(data2_path / 'final' / '*' / '*.xml'))
df2 = read_files2df(files)
files = glob(str(data1_path / 'final' / '*' / '*.xml'))
df3 = read_files2df(files)
total_xml_df = pd.concat([df, df2, df3],axis=0)
total_xml_df.index = [x.replace('.xml','.jpg') for x in total_xml_df.index]
total_xml_df.to_pickle('./all_xml_df.p')

In [6]:
total_xml_df = pd.read_pickle('./all_xml_df.p')
total_xml_df.head()

,image_size_depth,image_size_height,image_size_width,label,op_defect_name,path,pose,real_subfolder,subfolder,xmax,xmin,ymax,ymin
/home/jovyan/AOI/data/20180726_S1/final/fail/C02_115_000167.jpg,3.0,2048.0,1536.0,fail,C08,20180726_S1,Unspecified,fail,fail,905,655,905,1282
/home/jovyan/AOI/data/20180726_S1/final/fail/C02_115_000168.jpg,3.0,2048.0,1536.0,fail,C08,20180726_S1,Unspecified,fail,fail,276,197,276,1335
/home/jovyan/AOI/data/20180726_S1/final/fail/C02_200_031750.jpg,3.0,2048.0,1536.0,fail,C08,20180726_S1,Unspecified,fail,fail,702,600,702,1501
/home/jovyan/AOI/data/20180726_S1/final/fail/C02_204_003678.jpg,3.0,2048.0,1536.0,fail,C08,20180726_S1,Unspecified,fail,fail,648,605,648,1427
/home/jovyan/AOI/data/20180726_S1/final/fail/S07_200_031400.jpg,3.0,2048.0,1536.0,fail,C02,20180726_S1,Unspecified,fail,fail,702,600,702,1494


# subfolder is outdated. label == real_subfolder

In [20]:
(total_xml_df[['subfolder', 'real_subfolder','label', 'pose']]
            .groupby(['label','subfolder'])['pose']
            .count())


label  subfolder
fail   all/t_001     221
       all/t_002     103
       all/t_004       2
       fail          240
pass   all/t_001    2787
       all/t_002    1680
       all/t_003    1787
       all/t_004    1047
       all/t_011    1815
       all/t_016     742
       fail            1
Name: pose, dtype: int64

In [22]:
(total_xml_df[['subfolder', 'real_subfolder','label', 'pose']]
            .groupby(['label','real_subfolder'])['pose']
            .count())


label  real_subfolder
fail   fail               566
pass   pass              9859
Name: pose, dtype: int64

# collect all final_csv

In [23]:
data1_table = pd.read_csv(data1_path / 'final.csv')
data2_table = pd.read_csv(data2_path / 'final.csv')
data3_table = pd.read_csv(data3_path / 'final.csv')
data1_table['path'] = '20180703'
data2_table['path'] = '20180705'
data2_table['jpg_dir'] = data2_table['newfile'].apply(lambda x: x.split('/')[0])
data2_table['jpg_name'] = data2_table['newfile'].apply(lambda x: x.split('/')[1])
data3_table['path'] = '20180726_S1'
data_table = pd.concat([data1_table, data2_table, data3_table], axis=0, sort=False)


In [24]:
from pathlib import Path
data_table['subfolder'] = 'pass'
data_table.loc[data_table['label'] == 0, 'subfolder'] = 'fail'
data_table['image_filename'] = ('/home/jovyan/AOI/data/' + data_table['path'] + 
                                '/final/' + data_table['subfolder'] + '/' +
                                data_table['jpg_name'])


In [25]:
import os
file_exist = []
for row in data_table.iterrows():
    file_exist.append(os.path.isfile(row[1]['image_filename']))
data_table['file_exist'] = file_exist

In [26]:
data_table.set_index('image_filename', inplace=True)

In [27]:
if 'new_sub_part' in data_table.columns:
    data_table.drop(columns='new_sub_part', inplace=True)
data_table.to_pickle('./all_finalcsv.p')

In [28]:
total_xml_df = pd.read_pickle('./all_xml_df.p')
total_xml_df.drop(columns=['path','label', 'real_subfolder','pose', 'subfolder'],inplace=True)
total_xml_df.rename(columns={'xmax':'X2', 'xmin':'X1', 'ymin': 'Y1', 'ymax':'Y2'}, inplace=True)

In [29]:
data_table = pd.read_pickle('./all_finalcsv.p')
data_table = data_table.join(total_xml_df,lsuffix='_csv',rsuffix='_xml', on='image_filename', how='inner')
data_table.to_pickle('./all_finalcsv.p')

# find blue light images

In [ ]:


for row in tqdm(data_table.iterrows()):
    img = cv2.imread(row[0],1)
    rgb = img.sum(axis=0).sum(axis=0) / img.shape[0]/img.shape[1]
    data_table.loc[row[0],'ch0'] = rgb[0]
    data_table.loc[row[0],'ch1'] = rgb[1]
    data_table.loc[row[0],'ch2'] = rgb[2]
    

In [ ]:
data_table['blue_light_rough'] = True
data_table.loc[data_table['ch0'] < 50, 'blue_light_rough'] = False
data_table.to_pickle('./all_finalcsv.p')

In [7]:
data_table = pd.read_pickle('./all_finalcsv.p')

# group images according to their defect type

In [12]:
files_dict = {}
for r in data_table['op_defect_name'].unique():
    index = ((data_table['label']==0) &
             (data_table['op_defect_name'] == r) &
             (data_table['blue_light_rough']))
    if np.sum(index) > 0:
        files_dict[r] = data_table[index].index

In [16]:
import shutil
import os
files_dict['C02'][0].split('/')[-4]

'20180703'

In [19]:
base = Path('./failed')
base.mkdir(exis)

PosixPath('failed')

In [20]:
base = Path('./failed')
base.mkdir(exist_ok=True)
for k in tqdm(files_dict.keys()):
    p = base / k
    p.mkdir(exist_ok=True)
    for i,f in enumerate(files_dict[k]):
        date = f.split('/')[-4]
        shutil.copy(f, p / f"{date}.{f.split('/')[-1]}")
        f2 = f.replace('.jpg','.xml')
        shutil.copy(f2, p / f"{date}.{f2.split('/')[-1]}")

### passed reference images

In [21]:
index = (data_table['label']==1) & (data_table['op_defect_name'] == r) & (data_table['blue_light_rough'])
files = data_table[index].index.tolist()
np.random.shuffle(files)



In [22]:
base = Path('./passed')
files_passed_dict = {}
base.mkdir(exist_ok=True)
for k in tqdm(files_dict.keys()):
    p = base / k
    p.mkdir(exist_ok=True)
    index = ((data_table['label']==1) &
             (data_table['op_defect_name'] == k) &
             (data_table['blue_light_rough']))
    if np.sum(index) > 0:
        files = data_table[index].index.tolist()
        np.random.shuffle(files)
        files_passed_dict[k] = files[:550]
    if (k in files_passed_dict):
        for i,f in enumerate(files_passed_dict[k]):
            date = f.split('/')[-4]
            shutil.copy(f, p / f"{date}.{f.split('/')[-1]}")
            f2 = f.replace('.jpg','.xml')
            shutil.copy(f2, p / f"{date}.{f2.split('/')[-1]}")  
    else:
        print(k, ' no passed')

C01  no passed

